In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.optimize import Bounds
import seaborn as sns
import statistics
from tqdm import tqdm

In [3]:
def log_beta(beta0, i, c=1):
    beta = np.log(np.exp(c*beta0)+i)/c
    return beta

In [5]:
bounds = Bounds([0], [np.inf])
def langevin(x0, d, func, grad, maxiter = 100, eta0 = 0.001, beta0 = 1, beta_schedule = log_beta, c=1):
    """
    This code implements Gradient Langevin Algorithm with exact linesearch on the step size eta

    Args:
        x0 (numpy darray): initial point
        d (int): dimension of the objective function
        func (Callable): objective function
        grad (Callable): gradient of objective function
        maxiter (int): maximum number of iterations
        eta0 (float): initial value for eta
        beta0 (float): initial value for beta
        beta_schedule (Callable): annealing schedule for temperature, starting with beta0 and ending with beta1
        c (float): constant in logarithmic annealing schedule

    Output:
        f_list (numpy darray): the list of function values for each iteration
        x_list (numpy darray): the list of x values for each iteration
    """
    x_list = np.zeros((maxiter,d))
    x_list[0,:] = x0
    f_list = np.zeros((maxiter,))
    f_list[0] = func(x0)
    for i in range(1,maxiter):
        epsilon = np.random.normal(0, 1, d)
        beta = beta_schedule(beta0, i, c=c)
        def objective_function(eta):
            return func(x_list[i-1,:]- eta*grad(x_list[i-1,:]) + np.sqrt(2*eta/beta)*epsilon)
        # perform exact linesearch
        result = minimize(objective_function, eta0, method = "SLSQP", bounds=bounds)
        eta = result.x
        x_list[i,:] = x_list[i-1,:] - eta*grad(x_list[i-1,:]) + np.sqrt(2*eta/beta)*epsilon
        f_list[i] = func(x_list[i,:])
    return f_list, x_list

In [7]:
def gradient_descent(x0, d, func, grad, maxiter = 100, eta0 = 0.001):
    """
    This code implements Gradient Descent with exact linesearch on the step size eta

    Args:
        x0 (numpy darray): initial point
        d (int): dimension of the objective function
        func (Callable): objective function
        grad (Callable): gradient of objective function
        maxiter (int): maximum number of iterations
        eta0 (float): initial value for eta

    Output:
        f_list (numpy darray): the list of function values for each iteration
        x_list (numpy darray): the list of x values for each iteration
    """
    x_list = np.zeros((maxiter,d))
    x_list[0,:] = x0
    f_list = np.zeros((maxiter,))
    f_list[0] = func(x0)
    for i in range(1,maxiter):
        def objective_function(eta):
            return func(x_list[i-1,:]- eta*grad(x_list[i-1,:]))
        # perform exact linesearch
        result = minimize(objective_function, eta0, method = "SLSQP", bounds=bounds)
        eta = result.x
        x_list[i,:] = x_list[i-1,:] - eta*grad(x_list[i-1,:])
        f_list[i] = func(x_list[i,:])
    return f_list, x_list

In [9]:
def langevin_agd_x_noise(x0, d, func, grad, theta, maxiter = 100, L = None, C1 = 1):
    """
    This code implements Accelerated Langevin with Perturbed x

    Args:
        x0 (numpy darray): initial point
        d (int): dimension of the objective function
        func (Callable): objective function
        grad (Callable): gradient of objective function
        maxiter (int): maximum number of iterations
        theta (float): momentum parameter
        L (float): Lipschitz constant
        C1 (float): constant for scaling beta

    Output:
        f_list (numpy darray): the list of function values for each iteration
        x_list (numpy darray): the list of x values for each iteration
    """
    x_list = np.zeros((maxiter,d))
    y_list = np.zeros((maxiter,d))
    f_list = np.zeros((maxiter,))
    x_list[0,:] = x0
    f_list[0] = func(x_list[0,:])
    for i in range(maxiter-1):
        eps = np.random.normal(0, 1, size = (d,))
        eta = 1/L
        beta = C1*(i+1)**(2/3)
        xi = x_list[i,:] + np.sqrt(2*eta/beta)*eps
        y_list[i+1,:] = xi - eta*grad(xi)
        x_list[i+1,:] = y_list[i+1,:] + (1-theta)*(y_list[i+1,:]-y_list[i,:])
        f_list[i+1] = func(x_list[i+1,:])
    return f_list, x_list

In [32]:
def langevin_agd_y_noise(x0, d, func, grad, theta, maxiter = 100, L = None, C1 = 1):
    """
    This code implements Accelerated Langevin with Perturbed y

    Args:
        x0 (numpy darray): initial point
        d (int): dimension of the objective function
        func (Callable): objective function
        grad (Callable): gradient of objective function
        maxiter (int): maximum number of iterations
        theta (float): momentum parameter
        L (float): Lipschitz constant
        C1 (float): constant for scaling beta

    Output:
        f_list (numpy darray): the list of function values for each iteration
        x_list (numpy darray): the list of x values for each iteration
    """
    x_list = np.zeros((maxiter,d))
    y_list = np.zeros((maxiter,d))
    f_list = np.zeros((maxiter,))
    x_list[0,:] = x0
    f_list[0] = func(x_list[0,:])
    for i in range(maxiter-1):
        eps = np.random.normal(0, 1, size = (d,))
        eta = 1/L
        beta = C1*(i+1)**(2/3)
        y_list[i+1,:] = x_list[i,:] - eta*grad(x_list[i,:]) + np.sqrt(2*eta/beta)*eps
        x_list[i+1,:] = y_list[i+1,:] + (1-theta)*(y_list[i+1,:]-y_list[i,:])
        f_list[i+1] = func(x_list[i+1,:])
    return f_list, x_list

In [36]:
def nesterov(x0, d, func, grad, theta, maxiter = 100, eta = 0.01):
    """
    This code implements Nesterov's Accelerated Gradient

    Args:
        x0 (numpy darray): initial point
        d (int): dimension of the objective function
        func (Callable): objective function
        grad (Callable): gradient of objective function
        theta (float): momentum parameter
        maxiter (int): maximum number of iterations
        eta (float): step size

    Output:
        f_list (numpy darray): the list of function values for each iteration
        x_list (numpy darray): the list of x values for each iteration
    """
    x_list = np.zeros((maxiter,d))
    y_list = np.zeros((maxiter,d))
    f_list = np.zeros((maxiter,))
    x_list[0,:] = x0
    f_list[0] = func(x_list[0,:])
    for i in range(maxiter-1):
        # theta = i/(i+3)
        y_list[i+1,:] = x_list[i,:] - eta*grad(x_list[i,:])
        x_list[i+1,:] = y_list[i+1,:] + (1-theta)*(y_list[i+1,:]-y_list[i,:])
        f_list[i+1] = func(x_list[i+1,:])
    return f_list, x_list

In [40]:
def random_orthogonal_matrix(n):
    # Generate a random matrix
    A = np.random.randn(n, n)
    
    # QR decomposition
    Q, R = np.linalg.qr(A)
    
    return Q

In [42]:
Q = random_orthogonal_matrix(2)
D = np.diag([1,1])
H = Q @ D @ Q.T
g = np.random.randn(2,1)
def quadratic_func_well_cond(x):
    x = x.reshape((2,1))
    return 0.5*(x.T@H@x) + g.T@x

def quadratic_grad_well_cond(x):
    x = x.reshape((2,1))
    gf = H@x + g
    gf = gf.reshape((2,))
    return gf

In [44]:
Q = random_orthogonal_matrix(2)
D = np.diag([1e4,1])
H = Q @ D @ Q.T
g = np.random.randn(2,1)
def quadratic_func_ill_cond(x):
    return 0.5*(x.T@H@x) + g.T@x

def quadratic_grad_ill_cond(x):
    x = x.reshape((2,1))
    gf = H@x + g
    gf = gf.reshape((2,))
    return gf